# Ein Trainingsprojekt zur linearen Regression: Kundendaten im E-Commerce

## 1. Daten vorbereiten Teil A
### 1.1 Daten laden

In [1]:
import pandas as pd
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

In [2]:
customers = pd.read_csv("data/Ecommerce Customers.csv")
customers

,Email,Address,Avatar,Avg. Session Length,Time on App,Time on Website,Length of Membership,Yearly Amount Spent
0,mstephenson@fernandez.com,"835 Frank Tunnel\nWrightmouth, MI 82180-9605",Violet,34.497268,12.655651,39.577668,4.082621,587.951054
1,hduke@hotmail.com,"4547 Archer Common\nDiazchester, CA 06566-8576",DarkGreen,31.926272,11.109461,37.268959,2.664034,392.204933
2,pallen@yahoo.com,"24645 Valerie Unions Suite 582\nCobbborough, D...",Bisque,33.000915,11.330278,37.110597,4.104543,487.547505
3,riverarebecca@gmail.com,"1414 David Throughway\nPort Jason, OH 22070-1220",SaddleBrown,34.305557,13.717514,36.721283,3.120179,581.852344
4,mstephens@davidson-herman.com,"14023 Rodriguez Passage\nPort Jacobville, PR 3...",MediumAquaMarine,33.330673,12.795189,37.536653,4.446308,599.406092
...,...,...,...,...,...,...,...,...
495,lewisjessica@craig-evans.com,"4483 Jones Motorway Suite 872\nLake Jamiefurt,...",Tan,33.237660,13.566160,36.417985,3.746573,573.847438
496,katrina56@gmail.com,"172 Owen Divide Suite 497\nWest Richard, CA 19320",PaleVioletRed,34.702529,11.695736,37.190268,3.576526,529.049004
497,dale88@hotmail.com,"0787 Andrews Ranch Apt. 633\nSouth Chadburgh, ...",Cornsilk,32.646777,11.499409,38.332576,4.958264,551.620145
498,cwilson@hotmail.com,"680 Jennifer Lodge Apt. 808\nBrendachester, TX...",Teal,33.322501,12.391423,36.840086,2.336485,456.469510


### 1.2 Fehlende Werte prüfen/behandeln

In [3]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Email                 500 non-null    object 
 1   Address               500 non-null    object 
 2   Avatar                500 non-null    object 
 3   Avg. Session Length   500 non-null    float64
 4   Time on App           500 non-null    float64
 5   Time on Website       500 non-null    float64
 6   Length of Membership  500 non-null    float64
 7   Yearly Amount Spent   500 non-null    float64
dtypes: float64(5), object(3)
memory usage: 31.4+ KB


Es sind keine fehlenden Werte vorhanden, daher müssen wir nichts weiter unternehmen.

## 2. Explorative Datenanalyse (EDA)
### 2.1 Frage:
- Wie hängen die auf den Plattformen verbrachten Zeiten der Kunden mit ihren jährlichen Ausgaben zusammen?

In [4]:
fig_1 = px.scatter(
    customers,
    x="Time on Website",
    y="Yearly Amount Spent",
    color="Length of Membership",            
    size="Avg. Session Length",   
    marginal_x="histogram",
    marginal_y="histogram",
    opacity=0.5,
    size_max=8,
    title="Website-Zeit vs. Jahresausgaben"
)

fig_1.update_traces(marker=dict(line=dict(width=0.5, color="DarkSlateGrey")))
fig_1.update_layout(
    title_font=dict(size=18, family="Arial"),
    xaxis_title="Durchschnittliche Zeit auf der Website (Minuten)",
    yaxis_title="Jährliche Ausgaben ($)",
    template="plotly_white",          
    coloraxis_colorbar=dict(title="Dauer der <br>Mitgliedschaft (Minuten)")
)

fig_1

Die Punkte sind ziemlich gleichmäßig gestreut. Es gibt keine klare Tendenz, dass mehr Zeit auf der Website automatisch zu höheren Ausgaben führt. Kunden mit langer oder kurzer Website-Nutzung geben ähnliche Beträge im Jahr aus. Die reine Nutzungszeit auf der Website scheint kein guter Prädiktor (Voraussager) für die Ausgaben zu sein.

In [5]:
fig_2 = px.scatter(
    customers,
    x="Time on App",
    y="Yearly Amount Spent",
    color="Length of Membership",            
    size="Avg. Session Length",   
    marginal_x="histogram",
    marginal_y="histogram",
    opacity=0.5,
    size_max=8,
     title="App-Zeit vs. Jahresausgaben"
)

fig_2.update_traces(marker=dict(line=dict(width=0.5, color="DarkSlateGrey")))
fig_2.update_layout(
    title_font=dict(size=18, family="Arial"),
    xaxis_title="Durchschnittliche Zeit auf der App (Minuten)",
    yaxis_title="Jährliche Ausgaben ($)",
    template="plotly_white",          
    coloraxis_colorbar=dict(title="Dauer der <br>Mitgliedschaft (Minuten)")
)

fig_2

Hier erkennt man zumindest eine leichte positive Tendenz: Mit steigender App-Nutzung neigen Kunden dazu, mehr auszugeben. Der Zusammenhang ist aber nicht sehr stark – es ist mehr ein schwacher Trend als eine klare Linie. Die App-Nutzung könnte etwas mehr Einfluss auf die Ausgaben haben als die Website-Nutzung, möglicherweise weil der Kaufprozess dort einfacher oder zielgerichteter ist.

In beiden Plots sieht man: Kunden mit längerer Mitgliedschaft (gelbe/orange Punkte) geben tendenziell mehr Geld aus. Das ist eine starke Korrelation, die man schon visuell erkennen kann. Die Mitgliedsdauer ist vermutlich der wichtigste Einflussfaktor auf die Ausgaben – deutlich stärker als die Nutzungsdauer auf Website oder App.

### 2.2 Frage
- Welche Muster und Korrelationen lassen sich zwischen den Kundenmerkmalen erkennen, wenn man alle Variablen gleichzeitig betrachtet?

In [6]:
fig3 = px.scatter_matrix(
    customers,
    dimensions=["Avg. Session Length", "Time on App", "Time on Website", "Length of Membership", "Yearly Amount Spent"],
    color="Length of Membership",
    opacity=0.5
)
fig3.update_traces(diagonal_visible=True) 
fig3.update_layout(
    title="Scatter-Matrix der Kundenmerkmale",
    width=1200,
    height=1000,
)

fig3

In der Korrelationsmatrix spiegelt sich unsere Annahme wieder: Das Merkmal Length of Membership hat die höchste positive Korrelation mit Yearly Amount Spent. Die Problematik ist sehr gut für ein lineares Regressionsmodell geeignet.

## 3. Daten vorbereiten Teil B
### 3.1 Aufteilen der Daten in Trainings- und Testdaten

In [7]:
# Feature-Matrix:
X = customers.drop("Yearly Amount Spent", axis=1)

# Label-Vektor:
y = customers["Yearly Amount Spent"]

# Aufteilen der Daten:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### 3.2 Ausreißerprüfung (outlier detection/treatment)
Für uns sind nur die numerischen Features relevant:

- Avg. Session Length
- Time on App
- Time on Website
- Length of Membership
- Yearly Amount Spent

In [8]:
# Nur numerische Features:
num_cols = X_train.select_dtypes(include="number").columns
num_cols

Index(['Avg. Session Length', 'Time on App', 'Time on Website',
       'Length of Membership'],
      dtype='object')

In [9]:
import sys, os
sys.path.insert(0, os.path.abspath(".."))
from outliers import compute_iqr_report_df

iqr_info = compute_iqr_report_df(X_train, num_cols)
iqr_info

,Q1,Q3,IQR,lower,upper,n_outliers
Avg. Session Length,32.335,33.737,1.402,30.233,35.839,2
Time on App,11.394,12.760,1.366,9.344,14.809,3
Time on Website,36.312,37.684,1.373,34.253,39.743,1
Length of Membership,2.956,4.142,1.186,1.177,5.921,12


In [10]:
from outliers import plot_box_outliers_matrix

fig5, _ = plot_box_outliers_matrix(
    X_train, columns=num_cols, bounds=iqr_info,
    title="Boxplots mit Ausreißer (X_train)"
)

fig5

Wir sehen deutlich das Ausreißer vorhanden sind, diese müssen wir behandeln. Dazu erstelle nwir eine Funktion, mit der wir alle notwendigen Informationen über Ausreißer erhalten:

In [11]:
from outliers import iqr_clip_cols

X_train_clipped = iqr_clip_cols(df=X_train, columns=num_cols, bounds=iqr_info)
X_test_clipped = iqr_clip_cols(df=X_test, columns=num_cols, bounds=iqr_info)
X_train_clipped

,Email,Address,Avatar,Avg. Session Length,Time on App,Time on Website,Length of Membership
249,anntaylor@hotmail.com,"PSC 1634, Box 8167\nAPO AA 49814",Teal,33.780157,11.917636,36.844734,3.634996
433,efoster@williamson-boyd.org,"4968 Bennett Manors\nRogerschester, IL 81579-8333",Lavender,34.278248,11.822722,36.308545,2.117383
19,samuel46@love-west.net,544 Alexander Heights Suite 768\nNorth Johnvie...,LightSeaGreen,32.617856,13.989593,37.190504,4.064549
322,suzanne63@gmail.com,"229 Eric Mountains\nNew Caleb, PA 00396",LightSalmon,33.264632,10.732131,36.145792,4.086566
332,davisbriana@gmail.com,15298 Erickson Shore Apt. 056\nSouth Patrickfo...,MediumVioletRed,33.144234,11.737041,37.935189,2.190132
...,...,...,...,...,...,...,...
106,afry@ford.biz,"399 Jeremy Skyway Suite 377\nNorth Keithville,...",PaleTurquoise,32.291756,12.190474,36.152462,3.781823
270,jessicabrewer@simmons.net,"5052 Mccoy Passage Apt. 328\nTeresaport, FM 97139",LightBlue,34.006489,12.956277,38.655095,3.275734
348,qmorse@yahoo.com,"0360 Pearson Dam Suite 263\nLake Shawn, UT 659...",MediumTurquoise,31.812483,10.886921,34.897828,3.128639
435,sanchezamber@scott-patterson.com,"257 Hunt Manors\nSouth Charlottefort, CT 15033",Fuchsia,32.259973,14.132893,37.023479,3.762070


In [12]:
fig6, _ = plot_box_outliers_matrix(
    X_train_clipped, columns=num_cols, bounds=iqr_info,
    title="Boxplots mit Ausreißer (X_train)"
)

fig6

In [13]:
fig7, _ = plot_box_outliers_matrix(
    X_test_clipped, columns=num_cols, bounds=iqr_info,
    title="Boxplots mit Ausreißer (X_test)"
)

fig7

### 3.3 Verteilung der Daten prüfen
Jetzt können wir die Verteilung der kontinuierlichen Spalten begutachten. Dazu betrachten wir die sogenannte Skewness, diese Kenngröße beschreibt wie symmetrisch oder schief die Verteilung unserer Daten sind:

- 0: Symmetrische Verteilung (ähnlich Normalverteilung)
- > 0: Rechts-schiefe Verteilung
- < 0: Links-schiefe Verteilung

In [14]:
from eda import hist_skew_matrix

fig8, sk = hist_skew_matrix(X_train_clipped, columns=num_cols)
fig8

Unsere Daten haben eine symmetrische Verteilung, wir müssen also nichts unternehmen.

### 3.4 Skalierung / Standardisierung
In unserem Datensatz können Features sehr unterschiedliche Einheiten und Wertebereiche haben.

In [15]:
# Standardabweichung und Mittelwert vor Standardisierung prüfen:
for col in num_cols:
    mean_val = X_train_clipped[col].mean()
    std_val = X_train_clipped[col].std(ddof=0)
    print(f"{col}\nMittelwert = {mean_val:.4f}\nStandardabweichung = {std_val:.4f}")

Avg. Session Length
Mittelwert = 33.0629
Standardabweichung = 0.9807
Time on App
Mittelwert = 12.0607
Standardabweichung = 0.9837
Time on Website
Mittelwert = 37.0216
Standardabweichung = 1.0139
Length of Membership
Mittelwert = 3.5681
Standardabweichung = 1.0014


In [16]:
# 1) Scaler NUR auf dem TRAIN fitten
scaler = StandardScaler()
scaler.fit(X_train_clipped[num_cols].astype(float))

# 2) Kopien anlegen (Train/Test getrennt)
X_train_scaled = X_train_clipped.copy()
X_test_scaled  = X_test_clipped.copy()

# 3) Transform auf beide Datensätze anwenden
X_train_scaled[num_cols] = scaler.transform(X_train_clipped[num_cols].astype(float))
X_test_scaled[num_cols]  = scaler.transform(X_test_clipped[num_cols].astype(float))

In [17]:
# Standardabweichung und Mittelwert nach Standardisierung prüfen:
for col in num_cols:
    mean_val = X_train_scaled[col].mean()
    std_val = X_train_scaled[col].std(ddof=0)
    print(f"{col}\nMittelwert = {mean_val:.4f}\nStandardabweichung = {std_val:.4f}")

Avg. Session Length
Mittelwert = -0.0000
Standardabweichung = 1.0000
Time on App
Mittelwert = -0.0000
Standardabweichung = 1.0000
Time on Website
Mittelwert = -0.0000
Standardabweichung = 1.0000
Length of Membership
Mittelwert = -0.0000
Standardabweichung = 1.0000


### 3.5 Unnötige Features entfernen
Einige Features sind für uns nicht interessant, deswegen entfernen wir sie:

In [18]:
X_train_scaled

,Email,Address,Avatar,Avg. Session Length,Time on App,Time on Website,Length of Membership
249,anntaylor@hotmail.com,"PSC 1634, Box 8167\nAPO AA 49814",Teal,0.731344,-0.145428,-0.174471,0.066757
433,efoster@williamson-boyd.org,"4968 Bennett Manors\nRogerschester, IL 81579-8333",Lavender,1.239250,-0.241911,-0.703297,-1.448753
19,samuel46@love-west.net,544 Alexander Heights Suite 768\nNorth Johnvie...,LightSeaGreen,-0.453861,1.960768,0.166550,0.495714
322,suzanne63@gmail.com,"229 Eric Mountains\nNew Caleb, PA 00396",LightSalmon,0.205660,-1.350524,-0.863815,0.517702
332,davisbriana@gmail.com,15298 Erickson Shore Apt. 056\nSouth Patrickfo...,MediumVioletRed,0.082890,-0.329008,0.901009,-1.376104
...,...,...,...,...,...,...,...
106,afry@ford.biz,"399 Jeremy Skyway Suite 377\nNorth Keithville,...",PaleTurquoise,-0.786387,0.131919,-0.857237,0.213381
270,jessicabrewer@simmons.net,"5052 Mccoy Passage Apt. 328\nTeresaport, FM 97139",LightBlue,0.962136,0.910376,1.611028,-0.292007
348,qmorse@yahoo.com,"0360 Pearson Dam Suite 263\nLake Shawn, UT 659...",MediumTurquoise,-1.275105,-1.193176,-2.094642,-0.438898
435,sanchezamber@scott-patterson.com,"257 Hunt Manors\nSouth Charlottefort, CT 15033",Fuchsia,-0.818796,2.106437,0.001819,0.193655


In [19]:
X_train_scaled.drop(["Email", "Address", "Avatar"], axis=1, inplace=True)
X_test_scaled.drop(["Email", "Address", "Avatar"], axis=1, inplace=True)

### 4. Das richtige Modell auswählen
Für die Modellauswahl orientieren wir uns an der offiziellen Scikit-Learn-Map:

Dabei entscheiden wir uns für die Ridge Regression. Es ist ein Verfahren, das wie eine normale lineare Regression arbeitet, aber zusätzlich verhindert, dass einzelne Merkmale (Features) zu viel Einfluss bekommen. Sie bestraft im Training große Zahlen bei den Modell-Gewichten (Koeffizienten). Dadurch werden sehr Große, extreme Gewichte kleiner gemacht, wodurch das Modell "ruhiger" und stabiler wird. Ridge Regression ist wie eine normale Regression, aber mit einem "Dämpfer" auf den Gewichten, damit sie nicht zu verrückt spielt.



### 5. Modell trainieren
Nachdem die Daten vollständig vorbereitet wurden, kann das eigentliche Modelltraining beginnen. In diesem Schritt wird das zuvor ausgewählte Machine-Learning-Modell – in unserem Fall die Ridge Regression – auf die Trainingsdaten angepasst (fitting).

Damit das Modell möglichst gute Vorhersagen liefern kann, muss der wichtigste Hyperparameter alpha festgelegt werden. Er bestimmt die Stärke der Regularisierung:

- Wenn er klein ist, ähnelt das Modell einer normalen linearen Regression.
- Wenn er groß ist, hat man eine stärkere Glättung und weniger Varianz, aber möglicherweise höherer Bias.

In [20]:
from sklearn.linear_model import Ridge

ridge_basic = Ridge(alpha=1.0, random_state=42)
ridge_basic.fit(X_train_scaled, y_train)

,alpha,1.0
,fit_intercept,True
,copy_X,True
,max_iter,None
,tol,0.0001
,solver,'auto'
,positive,False
,random_state,42


### 6. Vorhersagen machen
Nachdem das Ridge-Regressionsmodell mit den Trainingsdaten trainiert wurde, kann es nun auf neue, unbekannte Daten angewendet werden. In unserem Fall sind das die zuvor zurückgehaltenen Testdaten.

Das Modell berechnet für jede Zeile im Testdatensatz einen geschätzten Verkaufspreis. Diese Vorhersagen lassen sich anschließend mit den tatsächlichen Werten vergleichen, um einen ersten Eindruck von der Modellgüte zu bekommen.

In [21]:
from evaluation import preview_regression_errors

y_pred_basic = ridge_basic.predict(X_test_scaled)

preview_regression_errors(y_test, y_pred_basic, n=10)

,Tatsächlicher Wert,Vorhergesagter Wert,Abweichung (Residuum),Abweichung (%),Absoluter Fehler,Absoluter Fehler (%),Quadratischer Fehler
361,401.033,400.840,0.193,0.048,0.193,0.048,0.037
73,534.777,543.140,-8.363,-1.564,8.363,1.564,69.939
374,418.603,425.701,-7.098,-1.696,7.098,1.696,50.379
155,503.978,501.387,2.591,0.514,2.591,0.514,6.716
104,410.070,408.022,2.047,0.499,2.047,0.499,4.191
394,557.608,570.613,-13.005,-2.332,13.005,2.332,169.125
377,538.942,531.688,7.254,1.346,7.254,1.346,52.627
124,514.337,505.874,8.463,1.645,8.463,1.645,71.621
68,408.620,406.036,2.584,0.632,2.584,0.632,6.676
450,475.015,472.903,2.113,0.445,2.113,0.445,4.464


### 7. Modell evaluieren
Die gängigen und aussagekräftigen Kennzahlen für ein Regressionsmodell sind:

- MAE (Mean Absolute Error): Durchschnittlicher absoluter Fehler
- RMSE (Root Mean Squared Error): Mittlerer quadratischer Fehler, große Ausreißer stärker gewichtet
- R² (Bestimmtheitsmaß): Erklärt den Anteil der Varianz, den das Modell erfasst

In [22]:
from evaluation import evaluate_regression

res = evaluate_regression(
    Ridge(),
    X_train_scaled, y_train,
    X_test_scaled, y_test,
    scoring="r2",
    with_train_metrics=True,
    extra_test_metrics=True,
    return_print=True
)

RMSE (Test): 11.1009
R² (Test): 0.9751
Adjusted R² (Test): 0.9741
MAE (Test): 9.0082 | MedAE: 8.4632 | MAPE: 1.89%
CV-R2 (Train)  Mittel: 0.9820 | Std: 0.0033 | Folds: 5
RMSE (Train):       10.6878
R² (Train):         0.9826


### 8. Pipeline erstellen
Nachdem alle notwendigen Schritte geklärt sind, ist es an der Zeite eine Scikit-Learn-Pipeline zu erzeugen.

In [ ]:
# importe 

import numpy as np
import pandas as pd

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression


# Daten laden falls noch nicht vorhanden mit ".read"

# 1. Ziel & Features festlegen
X = customers.drop(columns=["Yearly Amount Spent"])
y = customers["Yearly Amount Spent"]

# Train/Test-Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)

# 2. Einfacher Ausreißer-Clipper als Transformer
class IQRClipper(BaseEstimator, TransformerMixin):
    
    def __init__(self, factor=1.5):
        self.factor = factor

    def fit(self, X, y=None):
        X = np.asarray(X, dtype=float)
        q1 = np.nanpercentile(X, 25, axis=0)
        q3 = np.nanpercentile(X, 75, axis=0)
        iqr = q3 - q1

        lower = q1 - self.factor * iqr
        upper = q3 + self.factor * iqr

        # Konstante Spalten sauber behandeln (IQR==0)
        mask = (iqr == 0)
        lower[mask] = q1[mask]
        upper[mask] = q3[mask]

        self.lower_ = lower
        self.upper_ = upper
        return self

    def transform(self, X):
        X = np.asarray(X, dtype=float)
        return np.clip(X, self.lower_, self.upper_)
    
# 3. Preprocessing + Modell als Pipeline
# -> nur numerische Spalten verarbeiten (Textspalten werden ignoriert/gedroppt)
num_selector = make_column_selector(dtype_include=np.number)

preprocess = ColumnTransformer(
    transformers=[
        ("num", Pipeline(steps=[
            ("clip", IQRClipper(factor=1.5)),   # Ausreißer stutzen
            ("scale", StandardScaler())         # Standardisieren
        ]), num_selector)
    ],
    remainder="drop"  # nicht-numerische Spalten verwerfen (Email, Address, Avatar)
)

pipe = Pipeline(steps=[
    ("prep", preprocess),
    ("model", LinearRegression())
])

# 4. Training, CV und Test-Vorhersage
# Cross-Validation nur auf dem TRAIN-Set (leakage-sicher)
cv = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(pipe, X_train, y_train, scoring="r2", cv=cv)

# Fitte Pipeline auf dem kompletten TRAIN
pipe.fit(X_train, y_train)

# Test-Vorhersage
y_pred = pipe.predict(X_test)

# 5. Einfache Auswertung (Test-Set)
from evaluation import evaluate_regression

res = evaluate_regression(
    Ridge(),
    X_train_scaled, y_train,
    X_test_scaled, y_test,
    scoring="r2",
    with_train_metrics=True,
    extra_test_metrics=True,
    return_print=True
)
